## 데이터셋 준비하기
- 버킷 생성(단일 리전) 후 버킷에 데이터셋 upload (통으로 올려도 되고, 분할압축후 올리면 중간에 업로드 실패시 다시 업로드하기 편하다.)
- 버킷과 gcp로 실행한 jpyter notebook 연동
  - gcsfuse 로 마운트(colab의 드라이브 마운트와 같이 실시간 연동하는 식)
  - gsutill 로 접속하여 복사(cp) 하거나 다운로드를 통해 jpyter lab의 local로 옮겨서 사용(원본은 변화안됨, 딴사람의 버킷에 접속하여 다운가능)
- 가져온 압축파일 압축풀기

In [71]:
# for 문 진행상황 확인이 필요할떄 사용하기 위한 기능
from tqdm import tqdm

In [1]:
# jupyter폴더 아래에 dataset폴더를 생성합니다.
!mkdir /home/jupyter/dataset

In [2]:
# 앞에서 생성한 dataset폴더에 bukect 마운트하기
BUCKET_NAME = 'yolostudy'
MOUNT_PATH ='/home/jupyter/dataset' 
!gcsfuse --implicit-dirs {BUCKET_NAME} {MOUNT_PATH}
#!gcsfuse --implicit-dirs gne-dev-02-bucket /home/jupyter/dataset

{"time":"22/11/2023 06:09:07.196727","severity":"INFO","msg":"Start gcsfuse/1.2.0 (Go version go1.21.0) for app \"\" using mount point: /home/jupyter/dataset\n"}


In [3]:
import os
HOME = os.getcwd()
print(HOME)

/home/jupyter


### 버킷에 있는 데이터 로컬 주피터랩으로 복사하기

In [15]:
!df

Filesystem         1K-blocks     Used     Available Use% Mounted on
udev                30855224        0      30855224   0% /dev
tmpfs                6173212      448       6172764   1% /run
/dev/sdb1          102955604 33184200      65462596  34% /
tmpfs               30866060        0      30866060   0% /dev/shm
tmpfs                   5120        0          5120   0% /run/lock
/dev/sdb15            126678    10900        115778   9% /boot/efi
/dev/sda           205314024 41653332     163644308  21% /home/jupyter
yolostudy      1099511627776        0 1099511627776   0% /home/jupyter/dataset


### rsync = 파일 복사중에 진행상황 확인가능

In [19]:
## 파일 복사 진행상황 확인
!rsync -ah --info=progress2 '/home/jupyter/dataset/교통문제 해결을 위한 CCTV 교통 영상(고속도로)/' '/home/jupyter/highway/'

         53.09G 100%   42.63MB/s    0:19:47 (xfr#8319, to-chk=0/8381)   


In [20]:
!df

Filesystem         1K-blocks     Used     Available Use% Mounted on
udev                30855224        0      30855224   0% /dev
tmpfs                6173212      448       6172764   1% /run
/dev/sdb1          102955604 33184852      65461944  34% /
tmpfs               30866060        0      30866060   0% /dev/shm
tmpfs                   5120        0          5120   0% /run/lock
/dev/sdb15            126678    10900        115778   9% /boot/efi
/dev/sda           205314024 93514512     111783128  46% /home/jupyter
yolostudy      1099511627776        0 1099511627776   0% /home/jupyter/dataset


In [26]:
%cd /home/jupyter/~
!ls -al

/home/jupyter/~
total 20
drwxr-xr-x  5 jupyter jupyter 4096 Nov 22 06:45  .
drwxr-xr-x 14 jupyter jupyter 4096 Nov 22 07:07  ..
drwxr-xr-x  5 jupyter jupyter 4096 Nov 22 06:09  Training
drwxr-xr-x  5 jupyter jupyter 4096 Nov 22 06:32  Validation
drwxr-xr-x  4 jupyter jupyter 4096 Nov 22 06:09 '교통문제 해결을 위한 CCTV 교통 영상(고속도로)'


### 가져온 압축파일 압축풀기
- 훈련용 데이터, 검증용 데이터 전부 압축풀기
- 각각 이미지는 iamges 폴더, xml파일은 txt파일로 변환하여 labels 폴더로 저장

In [169]:
%%time
%cd ~/

import os
import zipfile
import multiprocessing
import concurrent.futures

cpu_num = os.cpu_count()  # 현재 pc의 cpu 코어 수 확인

## [라벨]1.수도권영동선 - 16MB = 1.07 s
# zipfilePath = 'highway/Training/bounding_box/[라벨]1.수도권영동선.zip'
# savePath = 'highway/Training/bounding_box/labels_total'

## [원천]1-1.수도권영동선 - 29.8GB = 56min 21s(cpu = 4) => 17min 7s(cpu = 14)
# zipfilePath = 'highway/Training/bounding_box/[원천]1-1.수도권영동선.zip'
# savePath = 'highway/Training/bounding_box/images_total'

## [라벨]1.수도권영동선 - 2.5MB = 6.27 s(cpu = 4) => 463 ms(cpu = 14)
# zipfilePath = 'highway/Validation/bounding_box/[라벨]1.수도권영동선.zip'
# savePath = 'highway/Validation/bounding_box/labels_total'

## [원천]1.수도권영동선 - 9GB =  18min 31s(cpu = 4) => 3min 31s(cpu = 14) 
## - 폴더 이름 깨짐 이슈때문에 jupyter local로 압축파일 복사하여 압축풀기 후 mv로 폴더이름 변경 진행함
# zipfilePath = 'highway/val/[원천]1.수도권영동선.zip'
# savePath = 'highway/val/images_total'

def unzip(file):
    with lock:
        zf.extract(file, path=savePath)

zf = zipfile.ZipFile(zipfilePath)    

m = multiprocessing.Manager()
lock = m.Lock()  # 여러개의 폴더의 압축을 풀떄 하나의 폴더의 압축을 다 풀때까지 다른작업 중지하기

with concurrent.futures.ProcessPoolExecutor(max_workers=cpu_num-2) as executor:
    executor.map(unzip, zf.infolist())  # map = 여러개의 기능을 동시에 실행

/home/jupyter
CPU times: user 3.07 s, sys: 1.1 s, total: 4.17 s
Wall time: 1min 28s


=================================

## 다운받은 데이터셋의 데이터를 전처리하기
- 폴더 내부의 파일 이름 목록 뽑아내기
- train/images 폴더를 생성하여 png 파일 전부 몰아넣기
- train/labels 폴더 생성 후 xml 파일을 txt파일로 변환한 데이터 전부 몰아넣기
- test, val 파일 모두 같은방식으로 변경

In [40]:
!pwd

/home/jupyter


In [129]:
# 가끔 아래의 에러가 발생할 경우 다음 코드를 실행한다.
# 로케일 에러
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [42]:
## png 파일과 변환시킨 txt 파일을 몰아서 담기위해 labels 폴더와 images 폴더 생성
!mkdir highway/train/images highway/train/highway/train/labels highway/val/images highway/val/labels

## 아래꺼 는 단순 확인용 코드 -> 실행 전에 경로 수정필요!

### train : iamges 데이터 파일명 골라내기 

In [82]:
# Trainging의 하위폴더에서 iamges_total 파일목록을 리스트로 만들기
import glob
trainPath = HOME + "/highway/Training/bounding_box/images_total"
train_iamage_file_list = glob.glob(trainPath + '/**', recursive=True)

print("경로가 포함된 리스트 수:",len(train_iamage_file_list))

# 경로명을 제외한 이름만 따로 뽑아 리스트에 다시 저장
train_iamage_file_name = [os.path.basename(x) for x  in train_iamage_file_list]  # bassename = 해당 경로를 생략하고 최종 파일 명만 출력
print("경로가 미포함된 리스트 수:",len(train_iamage_file_name))

경로가 포함된 리스트 수: 24270
경로가 미포함된 리스트 수: 24270


In [83]:
## trian_file_name 리스트에서 png 파일인 것만 초기화된 trian_file_list 리스트에 저장

train_iamage_file_list = []  # trian_file_list 리스트 초기화
for filename in train_iamage_file_name:
    if os.path.splitext(filename)[1] == '.png':
        train_iamage_file_list.append(filename)
        
print('폴더를 제외한 실제 파일 개수:', len(train_iamage_file_list))

폴더를 제외한 실제 파일 개수: 23951


In [84]:
# 제외된 폴더 개수 확인 
print(len(train_iamage_file_name) - len(train_iamage_file_list))

319


### train : labels 데이터 파일명 골라내기

In [65]:
# Trainging의 하위폴더에서 labels_total 파일목록을 리스트로 만들기
import glob
trainPath = HOME + "/highway/Training/bounding_box/labels_total"
train_label_file_list = glob.glob(trainPath + '/**', recursive=True)

print("경로가 포함된 리스트 수:",len(train_label_file_list))

# 경로명을 제외한 이름만 따로 뽑아 리스트에 다시 저장
train_label_file_name = [os.path.basename(x) for x  in train_label_file_list]  # bassename = 해당 경로를 생략하고 최종 파일 명만 출력
print("경로가 미포함된 리스트 수:",len(train_label_file_name))

## train_label_file_name 리스트의 파일목록은 xml 파일 형식임

경로가 포함된 리스트 수: 406
경로가 미포함된 리스트 수: 406


In [67]:
## train_label_file_name 리스트에서 xml 파일인 것만 초기화된 train_label_file_list 리스트에 저장

train_label_file_list = []  # train_label_file_list 리스트 초기화
for filename in train_label_file_name:
    if os.path.splitext(filename)[1] == '.xml':
        train_label_file_list.append(filename)
        
print('폴더를 제외한 실제 파일 개수:', len(train_label_file_list))

폴더를 제외한 실제 파일 개수: 405


In [86]:
# 제외된 폴더 개수 확인 
print(len(train_label_file_name) - len(train_label_file_list))

1


### validation : iamges 데이터 파일명 골라내기

In [79]:
# validation의 하위폴더에서 iamges_total 파일목록을 리스트로 만들기
import glob
valPath = HOME + "/highway/Validation/bounding_box/images_total"
val_iamage_file_list = glob.glob(valPath + '/**', recursive=True)

print("경로가 포함된 리스트 수:",len(val_iamage_file_list))

# 경로명을 제외한 이름만 따로 뽑아 리스트에 다시 저장
val_iamage_file_name = [os.path.basename(x) for x  in val_iamage_file_list]  # bassename = 해당 경로를 생략하고 최종 파일 명만 출력
print("경로가 미포함된 리스트 수:",len(val_iamage_file_name))

경로가 포함된 리스트 수: 8315
경로가 미포함된 리스트 수: 8315


In [80]:
## val_iamage_file_name 리스트에서 png 파일인 것만 초기화된 val_iamage_file_list 리스트에 저장

val_iamage_file_list = []  # val_iamage_file_list 리스트 초기화
for filename in val_iamage_file_name:
    if os.path.splitext(filename)[1] == '.png':
        val_iamage_file_list.append(filename)
        
print('폴더를 제외한 실제 파일 개수:', len(val_iamage_file_list))

폴더를 제외한 실제 파일 개수: 8112


In [85]:
# 제외된 폴더 개수 확인 
print(len(val_iamage_file_name) - len(val_iamage_file_list))

203


### validation : labels 데이터 파일명 골라내기

In [87]:
# validation의 하위폴더에서 labels_total 파일목록을 리스트로 만들기
import glob
valPath = HOME + "/highway/Validation/bounding_box/labels_total"
val_label_file_list = glob.glob(valPath + '/**', recursive=True)

print("경로가 포함된 리스트 수:",len(val_label_file_list))

# 경로명을 제외한 이름만 따로 뽑아 리스트에 다시 저장
val_label_file_name = [os.path.basename(x) for x  in val_label_file_list]  # bassename = 해당 경로를 생략하고 최종 파일 명만 출력
print("경로가 미포함된 리스트 수:",len(val_label_file_name))

경로가 포함된 리스트 수: 52
경로가 미포함된 리스트 수: 52


In [88]:
## val_label_file_name 리스트에서 xml 파일인 것만 초기화된 val_label_file_list 리스트에 저장

val_label_file_list = []  # train_label_file_list 리스트 초기화
for filename in val_label_file_name:
    if os.path.splitext(filename)[1] == '.xml':
        val_label_file_list.append(filename)
        
print('폴더를 제외한 실제 파일 개수:', len(val_label_file_list))

폴더를 제외한 실제 파일 개수: 51


In [89]:
# 제외된 폴더 개수 확인 
print(len(val_label_file_name) - len(val_label_file_list))

1


### train/validation 데이터의 png 파일 개수와 xml 파일 개수 출력

In [244]:
print("train 데이터의 파일 개수:", len(train_iamage_file_list))
print("train 데이터의 파일 개수:", len(train_label_file_list))
print("validation 데이터의 파일 개수:", len(val_iamage_file_list))
print("validation 데이터의 파일 개수:", len(val_label_file_list))

train 데이터의 파일 개수: 23951
train 데이터의 파일 개수: 405
validation 데이터의 파일 개수: 8112
validation 데이터의 파일 개수: 51


## train/images 폴더를 생성하여 png 파일 전부 몰아넣기

In [115]:
!pwd

/home/jupyter


In [116]:
# 폴더위치와 폴더 이름 yolo 규격에 맞는지 확인 후 위치 수정
# !mv highway/train/bounding_box/images highway/train/
# !mv highway/train/bounding_box/labels highway/train/

In [143]:
# !mv '/home/jupyter/highway/train/bounding_box/labels_total' '/home/jupyter/highway/train/labels_total'
# !mv '/home/jupyter/highway/train/bounding_box/images_total' '/home/jupyter/highway/train/images_total'
# !mv '/home/jupyter/highway/train/bounding_box/images' '/home/jupyter/highway/train/images'
# !mv '/home/jupyter/highway/train/bounding_box/labels' '/home/jupyter/highway/train/labels'
# !mv '/home/jupyter/highway/train/bounding_box/[라벨]1.수도권영동선.zip' '/home/jupyter/highway/train/[라벨]1.수도권영동선.zip'
# !mv '/home/jupyter/highway/train/bounding_box/[원천]1-1.수도권영동선.zip' '/home/jupyter/highway/train/[원천]1-1.수도권영동선.zip'
# !rm -rf /home/jupyter/highway/highway/train/bounding_box

# !mv '/home/jupyter/highway/val/bounding_box/labels_total' '/home/jupyter/highway/val/labels_total'
# !mv '/home/jupyter/highway/val/bounding_box/images_total' '/home/jupyter/highway/val/images_total'
# !mv '/home/jupyter/highway/val/bounding_box/images' '/home/jupyter/highway/val/images'
# !mv '/home/jupyter/highway/val/bounding_box/labels' '/home/jupyter/highway/val/labels'
# !mv '/home/jupyter/highway/val/bounding_box/[라벨]1.수도권영동선.zip' '/home/jupyter/highway/val/[라벨]1.수도권영동선.zip'
# !mv '/home/jupyter/highway/val/bounding_box/[원천]1.수도권영동선.zip' '/home/jupyter/highway/val/[원천]1.수도권영동선.zip'
# !rm -rf /home/jupyter/highway/val/bounding_box

In [144]:
dataPath = '/home/jupyter'

In [229]:
# train 개수 확인
a = glob.glob(dataPath+"/highway/train/images_total/*")
b = glob.glob(dataPath+"/highway/train/labels_total/*")
len(a), len(b)

(168, 168)

In [154]:
# valid 개수 확인
a = glob.glob(dataPath+"/highway/val/images_total/*")
b = glob.glob(dataPath+"/highway/val/labels_total/*")
len(a), len(b)

(51, 51)

## images_total 폴더 하위에 각 폴더명에 접근하여 내부 사진 파일들을 iamges 폴더에 몰아넣기

In [245]:
## 테스트 망했을 때 데이터 원복용 코드...
# !rm -rf highway/val/images_total
# !rm -rf highway/val/images
# !mkdir highway/val/images
# !mkdir highway/val/images_total

In [225]:
# images_total 하위의 폴더명 리스트를 전달받은 후,
# 해당 폴더 아래에 접근하여 폴더 내에 존재하는 이미지 파일들을 목적지(dstDir)로 옮기는 함수 선언
# 전달받을 parameter:
##  baseDir: /highway/val/images_total/폴더명
##  srcFiles: images_total/폴더/ 아래에 있는 이미지 파일들의 리스트
##  dstDir: 데이터를 이동시킬 최종 목적지 highway/val/images

def move_files(baseDir, srcFiles, dstDir):
    for srcFile in srcFiles:
        srcFilePath = os.path.join(baseDir, srcFile)  # 각각의 이미지데이터(png,jpg)의 전체 경로 저장
        #print(srcFilePath)
        #print(srcFilePath, dstDir)
        shutil.move(srcFilePath, dstDir) # 경로가 저장된 각각의 이미지파일을 롬기는 명령 실행

In [260]:
%%time
## image파일만 옮기기 실행, label 데이터는 xml파일을 txt파일로 변환하면서 저장 예정
## images_total 아래에 있는 각각의 폴더에 있는 png 파일을 전부 추출해 iamges폴더로 옮기기

## 초기 경로 지정

# val/iamges_total 데이터 옮기기 = 총 3333개
# srcBaseDir = os.path.join(dataPath, "highway/val/images_total")
# destPath = os.path.join(dataPath,"highway/val/images")

# tarin/images_total 데이터 옮기기 = 총 24102개
# srcBaseDir = os.path.join(dataPath, "highway/train/images_total")
# destPath = os.path.join(dataPath,"highway/train/images")


## =========== txt_total에 있는 txt 파일을 옮기기 전에 xnkTotxt 변환작업을 먼저 수행해야 함 ============ ##
## =========== 레이블xml2txt변환하기.ipynb 파일 참조 ============ ##

# val/txt_total 데이터 옮기기 = 총 3334 개
# srcBaseDir = os.path.join(dataPath, "highway/val/txt_total")
# destPath = os.path.join(dataPath,"highway/val/labels")

# train/txt_total 데이터 옮기기 = 총 24104개
#srcBaseDir = os.path.join(dataPath, "highway/train/txt_total")
#destPath = os.path.join(dataPath,"highway/train/labels")



# images_total 하위의 디렉토리 목록을 리스트 데이터로 생성
dirNames = os.listdir(srcBaseDir)

# 총 이동시킨 file의 개수를 확인하기 위해 사용
fileNumCounter=0

# images_total 하위의 디렉토리 목록 리스트를 하나씩 꺼내어 반복문 실행
for dirName in dirNames:
    srcBasePath = os.path.join(srcBaseDir,dirName)  # images_total/dirname경로 저장
    fileNames = os.listdir(srcBasePath)  # images_total/dirname경로 를 사용하여 images_total/dirname 폴더 내부의 이미지 파일목록을 리스트로 저장
    print("dir:{}, fileNum:{}".format(dirName, len(fileNames)))  # 폴더 이름과 각 폴더별 내부 이미지 데이터 개수 확인을 위해 출력
    fileNumCounter+=len(fileNames)  # 각 폴더의 내부 데이터 개수를 합산하기 위해 저장
    
    # 각 폴더명에 접근하여 내부 데이터를 옮기는 함수 호출
    move_files(srcBasePath, fileNames, destPath)
    #print(fileNames)

# 총 옮겨진 파일 개수 확인
print("fileNumCounter:{}".format(fileNumCounter))

dir:Suwon_CH02_20200720_2130_MON_9m_NH_highway_TW5_sunny_FHD, fileNum:150
dir:Suwon_CH04_20201012_1838_MON_9m_RH_highway_OW5_sunny_FHD, fileNum:150
dir:Suwon_CH04_20200721_1830_TUE_9m_RH_highway_OW5_sunny_FHD, fileNum:150
dir:Suwon_CH01_20201012_1723_MON_9m_RH_highway_TW5_sunny_FHD, fileNum:150
dir:Suwon_CH03_20201012_1933_MON_9m_RH_highway_OW5_sunny_FHD, fileNum:149
dir:Suwon_CH02_20200722_1730_WED_9m_NH_highway_TW5_sunny_FHD, fileNum:50
dir:Suwon_CH01_20200721_1700_TUE_9m_RH_highway_TW5_sunny_FHD, fileNum:150
dir:Suwon_CH04_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD, fileNum:150
dir:Suwon_CH04_20201010_1818_SAT_9m_RH_highway_OW5_sunny_FHD, fileNum:150
dir:Suwon_CH02_20200721_2030_TUE_9m_NH_highway_TW5_sunny_FHD, fileNum:100
dir:Suwon_CH03_20201011_1742_SUN_9m_RH_highway_OW5_sunny_FHD, fileNum:150
dir:Suwon_CH04_20200722_1600_WED_9m_NH_highway_OW5_rainy_FHD, fileNum:625
dir:Suwon_CH03_20200721_2000_TUE_9m_NH_highway_OW5_sunny_FHD, fileNum:100
dir:Suwon_CH03_20200722_1700_WED_9m_NH_

In [261]:
# 옮겨진 train/images 데이터와 val/images 데이터 개수 확인
len(os.listdir(destPath)), len(os.listdir('/home/jupyter/highway/val/images'))

(3334, 3333)

### 파일을 옮긴 후 빈 폴더 images_total, txt_total를 삭제

In [264]:
# !rm -rf /home/jupyter/highway/train/images_total /home/jupyter/highway/val/images_total
# !rm -rf /home/jupyter/highway/train/txt_total /home/jupyter/highway/val/txt_total

In [266]:
# train 데이터와 valid 데이터의 images, laebels 데이터 개수가 같은지 확인
# 현재 다르게 나옴

import glob
a = len(glob.glob(dataPath+"/highway/train/images/*"))
b = len(glob.glob(dataPath+"/highway/train/labels/*"))
c = len(glob.glob(dataPath+"/highway/val/images/*"))
d = len(glob.glob(dataPath+"/highway/val/labels/*"))
print('train 데이터 개수: image:{}, labels:{}'.format(a, b))
print('val 데이터 개수: image:{}, labels:{}'.format(c, d))

train 데이터 개수: image:24101, labels:24104
val 데이터 개수: image:3333, labels:3334


## GPT가 알려준 코드? -> 위의 move_folder 함수 사용으로 대체함

In [158]:
# !mv /home/jupyter/highway/val/images_total/*/* /home/jupyter/highway/val/images

In [159]:
# 빈껍데기 폴더 정리
# !rm -r /home/jupyter/highway/val/images/*/

### 위에코드가 리스트 목록이 크면 안되는 경우가 있다고 함, 안되면 실행 -> 위의 move_folder 함수 사용으로 대체함

In [160]:
# !find /home/jupyter/highway/train/labels/* -type f -exec mv -t /home/jupyter/highway/train/labels/ {} +

In [161]:
# !rm -r /home/jupyter/highway/train/labels/*/

## 파일명 일치 확인
- 파일명일치확인.ipynb 파일 참조

## 최종 전처리 결과 파일 개수가 일치하는것을 확인

In [268]:
# train 데이터와 valid 데이터의 images, laebels 데이터 개수가 같은지 확인
# 현재 다르게 나옴

import glob
a = len(glob.glob(dataPath+"/highway/train/images/*"))
b = len(glob.glob(dataPath+"/highway/train/labels/*"))
c = len(glob.glob(dataPath+"/highway/val/images/*"))
d = len(glob.glob(dataPath+"/highway/val/labels/*"))
print('train 데이터 개수: image:{}, labels:{}'.format(a, b))
print('val 데이터 개수: image:{}, labels:{}'.format(c, d))

train 데이터 개수: image:24101, labels:24101
val 데이터 개수: image:3333, labels:3333


## 이제 YOLO 모델 학습 시작해보기
- 고속도로CCTV데이터기반차량인식.ipynb 파일 참고